In [ ]:
def download_Project():
  !pip install gdown
  !gdown https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
  !unzip /content/Project_data.zip

download_Project()

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd

import random as rn
import numpy.random as nrn
import tensorflow as tf
import os

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Dropout, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

# import keras
# from keras.models import Sequential, Model
# from keras import layers
# from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
# from keras.layers import Conv3D, MaxPooling3D, Dropout, MaxPooling2D, GlobalAveragePooling2D
# from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
# from keras import optimizers

from keras.applications import ResNet101

import PIL
from PIL import Image
import cv2

import matplotlib.pyplot as plt

In [2]:
rn.seed(37)
nrn.seed(37)
tf.random.set_seed(37)

In [3]:
data_folder = "E:/Deep Learning/Gesture Recognition/Project_data"

train_df = pd.read_csv(f"{data_folder}/train.csv", sep=";", header=None)
train_df.columns = ['folder', 'type', 'class']
val_df = pd.read_csv(f"{data_folder}/val.csv", sep=";", header=None)
val_df.columns = ['folder', 'type', 'class']

In [4]:
def get_folder_list(df_csv):
  fptr = open(df_csv)
  folder_paths = fptr.readlines()
  fptr.close()
  return folder_paths

def get_batch_details(folder_list, batch_size):
  num_batches = len(folder_list) // batch_size
  folders_remain = len(folder_list) - (num_batches * batch_size)
  return num_batches, folders_remain

# def create_batch_folders(batch_size, img_ids, image_shape):
#          ## batch_images                                                                               ## batch_labels
#   return (np.zeros(shape = (batch_size, len(img_ids), image_shape[0], image_shape[1], 3)), np.zeros(shape = (batch_size, 5)))

def process_images(folder, folder_path, img_ids, image_shape, batch_images):
  imgs = os.listdir(f'{folder_path.split(";")[0]}')
  for idx, ind in enumerate(img_ids):
    ## read image
    image = cv2.imread(f'{folder_path.split(";")[0]}/{imgs[ind]}').astype(np.float32)
    ## resize image
    image = cv2.resize(image, (image_shape[0], image_shape[1]), interpolation = cv2.INTER_LINEAR)
    ## Seperating them into their respective R, G, B channels and normalizing the values
    ## Load labels for each image
    batch_images[folder, idx, :, :, 0] = image[:,:,0]/255.0   ## R
    batch_images[folder, idx, :, :, 1] = image[:,:,1]/255.0   ## G
    batch_images[folder, idx, :, :, 2] = image[:,:,2]/255.0   ## B

In [5]:
## path => folder path as string
## folder_list => list of folders in train/test folder as list
## batch_size => chosen batch size as int
## image_shape => as tuple (image_height, image_width)
def run_generator(path, folder_list, batch_size, image_shape):
  num_batches, folders_remain = get_batch_details(folder_list, batch_size)
  ## set how many images to view from total images of the video
  img_ids = list(range(0,30,2))
  while True:
    folder_list = np.random.permutation(folder_list)
    for batch in range(num_batches):
      # batch_images, batch_labels = create_batch_folders(batch_size, img_ids, image_shape)
      batch_images = np.zeros(shape = (batch_size, len(img_ids), image_shape[0], image_shape[1], 3))
      batch_labels = np.zeros(shape = (batch_size, 5))
      ## get folders for that batch
      for folder in range(batch_size):
        # process_images(folder, f'{path}/{folder_list[folder + (batch * batch_size)]}', img_ids, image_shape, batch_images)
      ## get images and labels for each folder in the batch
        imgs = os.listdir(f'{path}/{folder_list[folder + (batch * batch_size)].split(";")[0]}')
        for idx, ind in enumerate(img_ids):
            ## read image
            image = cv2.imread(f'{path}/{folder_list[folder + (batch * batch_size)].split(";")[0]}/{imgs[ind]}').astype(np.float32)
            ## resize image
            image = cv2.resize(image, (image_shape[0], image_shape[1]), interpolation = cv2.INTER_LINEAR)
            ## Seperating them into their respective R, G, B channels and normalizing the values
            ## Load labels for each image
            batch_images[folder, idx, :, :, 0] = image[:,:,0]/255.0   ## R
            batch_images[folder, idx, :, :, 1] = image[:,:,1]/255.0   ## G
            batch_images[folder, idx, :, :, 2] = image[:,:,2]/255.0   ## B
          ## Load labels for each image
        batch_labels[folder, int(folder_list[folder + (batch * batch_size)].strip().split(';')[2])] = 1
      yield batch_images, batch_labels
      
    ## Process the remaining folders
    # batch_images, batch_labels = create_batch_folders(folders_remain, img_ids, image_shape)
    if folders_remain > 0:
      batch_images = np.zeros(shape = (folders_remain, len(img_ids), image_shape[0], image_shape[1], 3))
      batch_labels = np.zeros(shape = (folders_remain, 5))
      folder_rem = folder_list[-folders_remain:]
      for folder in range(folders_remain):
      #   process_images(folder, f'{path}/{folder_rem[folder]}', img_ids, image_shape, batch_images)
        imgs = os.listdir(f'{path}/{folder_rem[folder].split(";")[0]}')
        for idx, ind in enumerate(img_ids):
            ## read image
            image = cv2.imread(f'{path}/{folder_rem[folder].split(";")[0]}/{imgs[ind]}').astype(np.float32)
            ## resize image
            image = cv2.resize(image, (image_shape[0], image_shape[1]), interpolation = cv2.INTER_LINEAR)
            ## Seperating them into their respective R, G, B channels and normalizing the values
            ## Load labels for each image
            batch_images[folder, idx, :, :, 0] = image[:,:,0]/255.0   ## R
            batch_images[folder, idx, :, :, 1] = image[:,:,1]/255.0   ## G
            batch_images[folder, idx, :, :, 2] = image[:,:,2]/255.0   ## B
        batch_labels[folder, int(folder_rem[folder].strip().split(';')[2])] = 1
      yield batch_images, batch_labels

In [7]:
# ## path => folder path as string
# ## folder_list => list of folders in train/test folder as list
# ## batch_size => chosen batch size as int
# ## image_shape => as tuple (image_height, image_width)
# def run_generator(path, folder_list, batch_size, image_shape):
#   num_batches, folders_remain = get_batch_details(folder_list, batch_size)
#   ## set how many images to view from total images of the video
#   img_ids = list(range(0,30,2))
#   while True:
#     folder_list = np.random.permutation(folder_list)
#     for batch in range(num_batches):
#       batch_images, batch_labels = create_batch_folders(batch_size, img_ids, image_shape)
#       ## get folders for that batch
#       for folder in range(batch_size):
#         process_images(folder, f'{path}/{folder_list[folder + (batch * batch_size)]}', img_ids, image_shape, batch_images)
#       ## get images and labels for each folder in the batch
#       #####################################
#           ## Load labels for each image
#         batch_labels[folder, int(folder_list[folder + (batch * batch_size)].strip().split(';')[2])] = 1
#       yield batch_images, batch_labels

#     ## Process the remaining folders
#     batch_images, batch_labels = create_batch_folders(folders_remain, img_ids, image_shape)
#     folder_rem = folder_list[-folders_remain:]
#     for folder in range(folders_remain):
#       process_images(folder, f'{path}/{folder_rem[folder]}', img_ids, image_shape, batch_images)
#       ####################################
#       batch_labels[folder, int(folder_rem[folder].strip().split(';')[2])] = 1
#     yield batch_images, batch_labels

In [6]:
curr_dt_time = datetime.now()

model_name = 'model_cnn3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_lr=0.001)
callbacks_list = [checkpoint, LR]

In [9]:
# train_folders = []
# val_folders = []
# for i in range(5):
#   train_folders.extend(train_df.loc[train_df['class'] == i, ['folder', 'class']][:20])
#   val_folders.extend(train_df.loc[train_df['class'] == i, ['folder', 'class']][-5:])

In [7]:
fptr = open(f'{data_folder}/train.csv')
train_paths = fptr.readlines()
fptr.close()
train_folder_list_ = np.random.permutation(train_paths)


fptr = open(f'{data_folder}/val.csv')
val_paths = fptr.readlines()
fptr.close()
val_folder_list_ = np.random.permutation(val_paths)


In [8]:
train_folder_list = train_folder_list_[:150]
val_folder_list = val_folder_list_[-50:]

In [12]:
# train_folder_list = np.random.permutation(train_folder_list)
# val_folder_list = np.random.permutation(val_folder_list)
# len(train_folder_list), len(val_folder_list)

In [9]:
batch_size = 10
num_epochs = 20
image_shape = (224, 224, 3)

In [10]:
if (len(train_folder_list)%batch_size) == 0:
    steps_per_epoch = int(len(train_folder_list)/batch_size)
else:
    steps_per_epoch = int(len(train_folder_list)//batch_size) + 1

if (len(val_folder_list)%batch_size) == 0:
    validation_steps = int(len(val_folder_list)/batch_size)
else:
    validation_steps = int(len(val_folder_list)//batch_size) + 1

In [15]:
len(train_folder_list), batch_size, len(train_folder_list)%batch_size, steps_per_epoch

(150, 10, 0, 15)

In [16]:
len(val_folder_list), batch_size, len(val_folder_list)%batch_size, validation_steps

(50, 10, 0, 5)

In [11]:
train_generator = run_generator(f'{data_folder}/train', train_folder_list, batch_size, image_shape)
val_generator = run_generator(f'{data_folder}/val', val_folder_list, batch_size, image_shape)

In [18]:
  # model = Sequential()

  # model.add(layers.Conv3D(64, (3, 3, 3), activation='relu', input_shape=vid_shape)),
  # model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  # model.add(layers.BatchNormalization()),

  # model.add(layers.Conv3D(128, (3, 3, 3), activation='relu')),
  # model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  # model.add(layers.BatchNormalization()),

  # model.add(layers.Conv3D(256, (3, 3, 3), activation='relu')),
  # model.add(layers.MaxPooling3D(pool_size=(1, 2, 2))),
  # model.add(layers.BatchNormalization()),

  # model.add(layers.Conv3D(512, (3, 3, 3), activation='relu')),

  # ## Flatten and Dense layers
  # model.add(layers.GlobalAveragePooling3D()),
  # model.add(layers.Dense(1024, activation='relu')),
  # model.add(layers.Dense(1024, activation='relu')),
  # model.add(layers.Dense(5, activation='softmax'))

In [12]:
num_classes = train_df['class'].unique().size

In [19]:
def cnn3D_model(vid_shape):
  model = Sequential()

  model.add(layers.Input(shape=vid_shape)),
  model.add(layers.Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')),
  model.add(layers.MaxPooling3D(pool_size=(2, 2, 2))),
  model.add(layers.BatchNormalization(axis=-1)),

  model.add(layers.Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')),
  # model.add(layers.MaxPooling3D(pool_size=(2, 2, 2))),
  # model.add(layers.BatchNormalization()),
  
  # model.add(layers.Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu')),
  # model.add(layers.MaxPooling3D(pool_size=(2, 2, 2))),
  # model.add(layers.BatchNormalization()),
  
  # model.add(layers.Conv3D(filters=256, kernel_size=(3, 3, 3), activation='relu')),
  # model.add(layers.MaxPooling3D(pool_size=(2, 2, 2))),
  # model.add(layers.BatchNormalization()),
  
  # model.add(layers.Conv3D(filters=512, kernel_size=(3, 3, 3), activation='relu')),

  ## Flatten and Dense layers
  model.add(layers.GlobalAveragePooling3D()),
  # model.add(layers.Flatten()),
  model.add(layers.Dense(512, activation='relu')),
  model.add(layers.Dense(num_classes, activation='softmax'))

  return model

In [22]:
model = cnn3D_model((15, 224, 224, 3))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_2 (Conv3D)               │ (None, 13, 222, 222,   │         5,248 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 6, 111, 111,    │             0 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 6, 111, 111,    │           256 │
│ (BatchNormalization)            │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 4, 109, 109,    │       110,656 │
│                                 │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling3d_1      │ (None, 64)             │             0 │
│ (GlobalAveragePooling3D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │        33,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 152,005 (593.77 KB)

 Trainable params: 151,877 (593.27 KB)

 Non-trainable params: 128 (512.00 B)

In [ ]:

model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, initial_epoch=0)

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 124s 7s/step - categorical_accuracy: 0.2601 - loss: 1.5843 - val_categorical_accuracy: 0.2300 - val_loss: 1.5983
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 334s 15s/step - categorical_accuracy: 0.3701 - loss: 1.4714 - val_categorical_accuracy: 0.2200 - val_loss: 1.6616
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 108s 6s/step - categorical_accuracy: 0.4447 - loss: 1.2723 - val_categorical_accuracy: 0.2000 - val_loss: 1.7086
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 86s 5s/step - categorical_accuracy: 0.5622 - loss: 1.0591 - val_categorical_accuracy: 0.2300 - val_loss: 1.6150
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 86s 5s/step - categorical_accuracy: 0.5225 - loss: 1.1304 - val_categorical_accuracy: 0.4100 - val_loss: 1.5203
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - categorical_accuracy: 0.5923 - loss: 0.9728 - val_categorical_accuracy: 0.4300 - val_loss: 1.4701
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - categorical_accuracy: 0.6230 - loss: 0.9234 

In [ ]:
print(tf.__version__)

In [ ]:
print(keras.__version__)

In [ ]:
train_generator = run_generator(f'{data_folder}/train', train_folder_list, batch_size, image_shape)

In [ ]:
print(next(train_generator)[0].shape, next(train_generator)[1].shape)
# print(next(train_generator)[0], next(train_generator)[1])

In [13]:
val_generator = run_generator(f'{data_folder}/val', val_folder_list, batch_size, image_shape)

In [ ]:
print(next(val_generator)[0].shape, next(val_generator)[1].shape)

In [ ]:
for i in range(30):
    print(next(train_generator)[0].shape, next(train_generator)[1].shape)

In [93]:
val_generator = run_generator(f'{data_folder}/val', val_folder_list, 4, image_shape)

In [ ]:
j = 0
for i in range(30):
    # print(next(val_generator)[0].shape, next(val_generator)[1].shape)
    if j%3 == 0:
        # print(next(val_generator)[1], end = '\n\n')
        print(next(val_generator)[0].shape, next(val_generator)[1].shape)
        # print(next(val_generator))

In [ ]:
val_folder_list

In [48]:
folder_list = ["WIN_20180925_18_01_40_Pro_Left_Swipe_new;Left_Swipe_new;0\n",
               "WIN_20180907_16_02_09_Pro_Thumbs Down_new;Thumbs Down_new;3\n",
               "WIN_20180907_16_05_10_Pro_Right Swipe_new;Right Swipe_new;1\n",
               "WIN_20180907_16_18_23_Pro_Thumbs Down_new;Thumbs Down_new;3\n",
               "WIN_20180925_17_38_43_Pro_Thumbs_Up_new;Thumbs_Up_new;4\n"]

In [204]:
## path => folder path as string
## folder_list => list of folders in train/test folder as list
## batch_size => chosen batch size as int
## image_shape => as tuple (image_height, image_width)
def run_generator_1(path, folder_list, batch_size, image_shape):
  num_batches, folders_remain = get_batch_details(folder_list, batch_size)
  ## set how many images to view from total images of the video
  img_ids = list(range(0,30,2))
  while True:
    folder_list = np.random.permutation(folder_list)
    for batch in range(num_batches):
      # batch_images, batch_labels = create_batch_folders(batch_size, img_ids, image_shape)
      batch_images = np.zeros(shape = (batch_size, len(img_ids), image_shape[0], image_shape[1], 3))
      batch_labels = np.zeros(shape = (batch_size, 5))
      ## get folders for that batch
      for folder in range(batch_size):
        # process_images(folder, f'{path}/{folder_list[folder + (batch * batch_size)]}', img_ids, image_shape, batch_images)
      ## get images and labels for each folder in the batch
        imgs = os.listdir(f'{path}/{folder_list[folder + (batch * batch_size)].split(";")[0]}')
        for idx, ind in enumerate(img_ids):
            ## read image
            image = cv2.imread(f'{path}/{folder_list[folder + (batch * batch_size)].split(";")[0]}/{imgs[ind]}').astype(np.float32)
            ## resize image
            image = cv2.resize(image, (image_shape[0], image_shape[1]), interpolation = cv2.INTER_LINEAR)
            ## Seperating them into their respective R, G, B channels and normalizing the values
            ## Load labels for each image
            batch_images[folder, idx, :, :, 0] = image[:,:,0]/255.0   ## R
            batch_images[folder, idx, :, :, 1] = image[:,:,1]/255.0   ## G
            batch_images[folder, idx, :, :, 2] = image[:,:,2]/255.0   ## B
          ## Load labels for each image
        batch_labels[folder, int(folder_list[folder + (batch * batch_size)].strip().split(';')[2])] = 1
      yield batch_images, batch_labels
      
    ## Process the remaining folders
    # batch_images, batch_labels = create_batch_folders(folders_remain, img_ids, image_shape)
    if folders_remain > 0:
      batch_images = np.zeros(shape = (folders_remain, len(img_ids), image_shape[0], image_shape[1], 3))
      batch_labels = np.zeros(shape = (folders_remain, 5))
      folder_rem = folder_list[-folders_remain:]
      for folder in range(folders_remain):
      #   process_images(folder, f'{path}/{folder_rem[folder]}', img_ids, image_shape, batch_images)
        imgs = os.listdir(f'{path}/{folder_rem[folder].split(";")[0]}')
        for idx, ind in enumerate(img_ids):
            ## read image
            image = cv2.imread(f'{path}/{folder_rem[folder].split(";")[0]}/{imgs[ind]}').astype(np.float32)
            ## resize image
            image = cv2.resize(image, (image_shape[0], image_shape[1]), interpolation = cv2.INTER_LINEAR)
            ## Seperating them into their respective R, G, B channels and normalizing the values
            ## Load labels for each image
            batch_images[folder, idx, :, :, 0] = image[:,:,0]/255.0   ## R
            batch_images[folder, idx, :, :, 1] = image[:,:,1]/255.0   ## G
            batch_images[folder, idx, :, :, 2] = image[:,:,2]/255.0   ## B
        batch_labels[folder, int(folder_rem[folder].strip().split(';')[2])] = 1
      print(batch_images.shape, batch_labels.shape)
      yield batch_images, batch_labels

In [140]:
train_folder_list = train_folder_list_[:150]
val_folder_list = val_folder_list_[-15:]

batch_size = 5
num_epochs = 20
image_shape = (224, 224, 3)

In [205]:
val_generator = run_generator_1(f'{data_folder}/val', val_folder_list, 5, image_shape)

In [ ]:
# j = 0
# for i in range(4):
#     # print(next(val_generator)[1], end = '\n\n')
#     # print(next(val_generator)[0].shape, next(val_generator)[1].shape)
#     next(val_generator)

next(val_generator)[0].shape, next(val_generator)[1].shape

In [ ]:
folders_remain = 3

folder_list[-folders_remain:]

In [50]:
###        Build a Model with CNN-GRU network

In [17]:
def gru_model(vid_shape):
    resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Create a sequential model to process the video data
    model = Sequential()

    # Use TimeDistributed to apply ResNet50 to each frame of the video
    model.add(TimeDistributed(resnet_model, input_shape=(15, 224, 224, 3)))

    # You can add a global pooling layer after ResNet50 (optional)
    model.add(TimeDistributed(GlobalAveragePooling2D()))

    # Add a GRU layer to process the sequence of features from each frame
    model.add(GRU(256))

    # Add a dense layer for classification (optional, depending on your task)
    model.add(Dense(5, activation='softmax'))  # Assuming 5 classes

    return model

In [ ]:
model = gru_model((224, 224, 3))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

In [ ]:

model.fit(
            train_generator, 
            steps_per_epoch = steps_per_epoch, 
            epochs = num_epochs, 
            verbose = 1,
            validation_data = val_generator,
            validation_steps = validation_steps, 
            class_weight = None, 
            initial_epoch = 0
          )

In [25]:
num_classes = 5  # Adjust according to your dataset

# Input shape: (timesteps, height, width, channels)
input_shape = (15, 224, 224, 3)  # 15 frames per sequence

# Load ResNet101 as the feature extractor
resnet101 = ResNet101(weights='imagenet', include_top=False)

# Freeze ResNet101 layers to prevent training
resnet101.trainable = False

# Build the model
model = Sequential([
    # TimeDistributed applies ResNet101 to each frame
    TimeDistributed(resnet101, input_shape=input_shape),
    TimeDistributed(GlobalAveragePooling2D()),  # Pooling spatial features
    GRU(256, return_sequences=False),  # GRU for temporal modeling
    Dropout(0.5),  # Regularization
    Dense(num_classes, activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

c:\Users\pkhli\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_1              │ (None, 15, 7, 7, 2048) │    42,658,176 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 15, 2048)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 256)            │     1,771,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,430,469 (169.49 MB)

 Trainable params: 1,772,293 (6.76 MB)

 Non-trainable params: 42,658,176 (162.73 MB)

In [26]:

model.fit(
            train_generator, 
            steps_per_epoch = steps_per_epoch, 
            epochs = num_epochs, 
            verbose = 1,
            validation_data = val_generator,
            validation_steps = validation_steps, 
            class_weight = None, 
            initial_epoch = 0
          )

Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 388s 18s/step - accuracy: 0.2171 - loss: 1.8786 - val_accuracy: 0.2200 - val_loss: 1.5833
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 176s 12s/step - accuracy: 0.2294 - loss: 1.7060 - val_accuracy: 0.2200 - val_loss: 1.5748
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 183s 12s/step - accuracy: 0.2086 - loss: 1.7955 - val_accuracy: 0.2000 - val_loss: 1.5808
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 175s 12s/step - accuracy: 0.1978 - loss: 1.7951 - val_accuracy: 0.2400 - val_loss: 1.5679
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 174s 12s/step - accuracy: 0.2420 - loss: 1.6922 - val_accuracy: 0.2200 - val_loss: 1.5524
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 173s 12s/step - accuracy: 0.2089 - loss: 1.6645 - val_accuracy: 0.3200 - val_loss: 1.5326
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 174s 12s/step - accuracy: 0.2695 - loss: 1.6557 - val_accuracy: 0.3000 - val_loss: 1.5132
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 176s 12s/step - accuracy: 0.2303 - loss: 1.6797 - val_accuracy: 0.

: 

In [ ]:
# vid_input = layers.Input(shape=(15, 224, 224, 3))
# resnet_model = ResNet101(
#                             include_top = False,
#                             weights = 'imagenet',
#                             pooling = None,
#                             input_tensor=layers.Input(shape=vid_input)
#                         )
# for layer in resnet_model.layers[:70]: layer.trainable = False
# for layer in resnet_model.layers[70:]: layer.trainable = True
# resnet_output = layers.GlobalAveragePooling2D()(resnet_model.output)
# cnn_model = keras.Model(inputs = resnet_model.input, outputs = resnet_output)
# cnn_model = layers.TimeDistributed(cnn_model)(vid_input)

# gru_model = layers.GRU(units = 256, return_sequences=True)(cnn_model)
# gru_model = layers.TimeDistributed(layers.Dense(num_classes, activation='softmax'))(gru_model)



# vid_input = (15, 224, 224, 3)
# resnet_model = ResNet101(
#                             include_top = False,
#                             weights = 'imagenet',
#                             pooling = None,
#                             input_shape=image_shape
#                         )
# for layer in resnet_model.layers[:70]: layer.trainable = False
# for layer in resnet_model.layers[70:]: layer.trainable = True

# resnet_output = layers.TimeDistributed(resnet_model, input_shape = vid_input)
# resnet_output = layers.TimeDistributed(layers.GlobalAveragePooling2D())(resnet_output)
# gru_model = layers.GRU(units = 256, return_sequences=True)(resnet_output)
# gru_model = layers.TimeDistributed(layers.Dense(num_classes, activation='softmax'))(gru_model)

# grumodel = keras.Model(inputs = resnet_model.input, outputs = gru_model)

c:\Users\pkhli\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: <TimeDistributed name=time_distributed_7, built=False> (of type <class 'keras.src.layers.rnn.time_distributed.TimeDistributed'>)

In [ ]:
def gru_model():
    model = Sequential()

    resnet_model = ResNet101(
        include_top=False,
        weights='imagenet',
        pooling=None,
        input_shape=(224, 224, 3)
    )

    for layer in resnet_model.layers[:65]:
        layer.trainable = False
    for layer in resnet_model.layers[65:]:
        layer.trainable = True

    model.add(layers.TimeDistributed(resnet_model, input_shape = (15, 224, 224, 3))),
    model.add(layers.TimeDistributed(layers.GlobalAveragePooling2D())),
    model.add(layers.GRU(units=64, return_sequences=False)),
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [52]:
# def gru_model():
#     model = Sequential()
#     model.add(layers.Input(shape=(15, 224, 224, 3))),

#     resnet_model = ResNet101(
#                                 include_top = False,
#                                 weights = 'imagenet',
#                                 pooling = None,
#                                 input_shape = (224, 224, 3)
#                              )
#     for layer in resnet_model.layers[:65]: layer.trainable = False
#     for layer in resnet_model.layers[65:]: layer.trainable = True

#     model.add(layers.TimeDistributed(resnet_model)),
#     model.add(layers.TimeDistributed(layers.GlobalAveragePooling2D())),
#     model.add(layers.GRU(units = 256, return_sequences=True))
#     model.add(layers.Dense(num_classes, activation='softmax'))

#     return resnet_model

In [17]:
def gru_model():
    model = Sequential()

    resnet_model = ResNet101(
        include_top=False,
        weights='imagenet',
        pooling=None,
        input_shape=(224, 224, 3)
    )

    for layer in resnet_model.layers[:65]:
        layer.trainable = False
    for layer in resnet_model.layers[65:]:
        layer.trainable = True

    model.add(layers.TimeDistributed(resnet_model, input_shape = (15, 224, 224, 3))),
    model.add(layers.TimeDistributed(layers.GlobalAveragePooling2D())),
    model.add(layers.GRU(units=2, return_sequences=False)),
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [18]:
model = gru_model()                                                                                  # keras.Model(resnet_model.input, outputs = gru_model)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed_3              │ (None, 15, 7, 7, 2048) │    42,658,176 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 15, 2048)       │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 2)              │        12,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            15 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,670,503 (162.78 MB)

 Trainable params: 41,463,335 (158.17 MB)

 Non-trainable params: 1,207,168 (4.60 MB)

In [ ]:
model.fit(
            train_generator, 
            steps_per_epoch = steps_per_epoch, 
            epochs = num_epochs, 
            verbose = 1,
            validation_data = val_generator,
            validation_steps = validation_steps, 
            class_weight = None, 
            initial_epoch = 0
          )

Epoch 1/20
